# Metallicity Switchboard

In [18]:
import astropy.units as u
import astropy.constants as ac
from astropy.cosmology import FlatLambdaCDM
import numpy as np
import pandas as pd
import os
from os.path import expanduser
from astropy.table import Table
from glob import glob
from O3N2 import*
from SFR import*
from Distance import*
from Ha_Luminosity import*
from findR23 import*
from O3HB import*

In [19]:
# gala is a data table of Final_galinfo.csv file which 
# is a subset of data from the galaxyinfo.xlsx data from CGM^2

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

galaxy = pd.read_csv('../test/Final_galinfo.csv')

galaxy.to_numpy()   # This turns the DataFrame into a NumPy array.

# This 'for' loop will look for '-99' values and change them to 'NaN' values.

columns = ['NII_flux', 'SIIa_flux', 'Halpha_flux', 'OII_flux', 'Hbeta_flux', 'OIII_flux',
          'SIIb_flux']

for i in columns:
    n = 0
    while n < len(galaxy):
        
        if galaxy[i][n] == -99.0:
            galaxy[i][n] = np.nan
        n += 1    
        
        
galaxy[0:2]

/Users/josephbreneman/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,specfile,id,OBJECT,cgm_name,gal_ra,gal_dec,z,OII_flux,OII_flux_sig,OII_LQ,Hbeta_flux,Hbeta_flux_sig,Hbeta_LQ,OIII_flux,OIII_flux_sig,OIII_LQ,Halpha_flux,Halpha_flux_sig,Halpha_LQ,NII_flux,NII_flux_sig,NII_LQ,SIIa_flux,SIIa_flux_sig,SIIa_LQ,SIIb_flux,SIIb_flux_sig,SIIb_LQ,Comment
0,J022601.51+001532.5.fits,J022601.51+001532.5._03,J0226+0015,J0226+0015_271_194,36.506302,0.25904,0.352715,NaN,-99.0,-99.0,2.130000e-17,2.440000e-18,1.0,2.250000e-17,1.760000e-18,1.0,3.120000e-17,2.470000e-18,1.0,1.120000e-17,2.570000e-18,1.0,1.290000e-17,7.990000e-19,1.0,1.760000e-18,1.140000e-18,1.0,NaN
1,J022602.41+001536.6.fits,J022602.41+001536.6._01,J0226+0015,J0226+0015_272_181,36.510059,0.26017,0.297832,NaN,-99.0,-99.0,NaN,-9.900000e+01,-99.0,2.190000e-17,1.870000e-18,1.0,2.080000e-17,8.410000e-19,1.0,NaN,-9.900000e+01,-99.0,NaN,-9.900000e+01,2.0,NaN,-9.900000e+01,2.0,NaN


## Calculate Distance

In [20]:
galaxy['Distance'] = Distance(galaxy['z'])

#galaxy

## Calculate Halpha and Hbeta Luminosity

In [21]:
galaxy['Ha_luminosity'] = Ha_luminosity(galaxy['Distance'],galaxy['Halpha_flux'])

Hbeta = 2.86*galaxy['Hbeta_flux']
galaxy['Hb_luminosity'] = Ha_luminosity(galaxy['Distance'],Hbeta)


#galaxy

## Calculate SFR

In [22]:
# Calculating SFR using Halpha luminosity
galaxy['SFR_Ha'] = SFR(galaxy['Ha_luminosity'])

# Calculating SFR using Hbeta luminosity, and then eliminating values (NaN) 
# that already have 'SFR_Ha' values not equal to NaN
sfr_b = SFR(galaxy['Hb_luminosity'])
nosfr_b = np.where(galaxy['SFR_Ha'] > 0.0)
sfr_b[nosfr_b] = np.nan
galaxy['SFR_Hb'] = sfr_b

#galaxy

In [23]:
SFR_switchboard(galaxy)

NameError: name 'SFR_flags' is not defined

## Calculate 'O3HB'

In [24]:
galaxy['O3HB'] = O3HB(galaxy['OIII_flux'],galaxy['Hbeta_flux'])

#galaxy

## Calculate 'N2' 

In [25]:
# Supplemental 'N2' function

def metal_Ha(F_N, F_Ha):
    
    '''
    This function will calculate the flux ratio of Nitrogen II (6583) and H_alpha
    
    F_N = flux of Nitrogen II
    F_Ha = Flux of H_alpha
    '''
    
    n2 = F_N / F_Ha
    
    return n2

galaxy['N2'] = metal_Ha(galaxy['NII_flux'],galaxy['Halpha_flux'])

#galaxy

## Calculate 'O3N2'

In [26]:
galaxy['O3N2'] = O3N2(galaxy)

#galaxy

## Calculate 'Z_O3N2L'

In [27]:
galaxy['Z_O3N2L'] = Z_O3N2L(galaxy)

#galaxy

#### -The 'O3N2' function only calculates metallicity if O3N2 < 1.9. There needs to be a function for calculating metallicity for O3N2 > 1.9.

In [28]:
np.where(galaxy['O3N2'] > 1.9)

(array([414]),)

-If 'O3N2' or 'O3HB' could not be calculated (NaN) and we only have N2 index then the equation for calculating 'Z_O3N2+' (when O3N2 > 1.9) can be used for calculating metallicity.

In [29]:
# This cell finds the N2 values that require metallicity calculations

n2_maybe = np.where((np.isnan(galaxy['O3HB']) & (np.isnan(galaxy['O3N2']))))

n2 = np.array(galaxy['N2'])
n2_yes = n2[n2_maybe]
np.where(n2_yes > 0.0)

/Users/josephbreneman/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys


(array([  0,  42,  45,  46,  48,  54,  62,  65,  66,  74,  75,  76,  89,
         91,  92,  93,  95,  99, 145, 163, 181, 201, 202, 206, 207, 213,
        214, 219, 223, 233, 234, 236, 243]),)

## Mass and friends

In [30]:
# Steven's technique

# filepath for table containing stellar masses for all the galaxies that we've been working with:

home = expanduser('~')
path = os.path.join(home,'Dropbox','**','cgmsq_allsurveys_cgmsystable.fits')
massfilepath = glob(path, recursive=True)

# Extracts table from fits file and resolves various issues required to concatenate this new table (which Matt
# made at some point previously) with our new SFR info-containing table.

mass_and_friends = Table.read(massfilepath[-1])
mass_and_friends.remove_columns(['zRR', 'zRR_Err', 'spectype_rr'])
mass_and_friends = mass_and_friends.to_pandas()
mass_and_friends['id'] = mass_and_friends['id'].str.decode('utf-8')

# merges mass table with initial (SFR-containing) one:

galaxy = pd.merge(galaxy, mass_and_friends, on=['id'], how='right')

# sorting out some (more) naming conflicts:

galaxy.rename(columns={'z_x': 'z'}, inplace=True)


## Calculate R23

In [31]:
r23 = findR23(galaxy['OII_flux'],galaxy['OIII_flux'],galaxy['NII_flux'],galaxy['Halpha_flux'],
             galaxy['Hbeta_flux'],galaxy['mstars'])

## Add columns for R23 metallicity

In [32]:
galaxy['Z_R23'] = r23[0]
galaxy['Z_R23_flag'] = r23[1]

In [34]:
# Defining the 'Z_flag' column

z_flag = np.array((len(galaxy))*[0])

# Flagging metallicity calculated with O3N2
one = np.where(galaxy['Z_O3N2L'] > 0.0) 
z_flag[one] = 1

#two = requires N2 index metallicity calculations

# Flagging metallicity calculated with R23
three = np.where(galaxy['Z_R23'] > 0.0)
z_flag[three] = 3

galaxy['Z_Flag'] = z_flag

#galaxy

-Save to .csv